# Assignment 8
## Reconstruction of image using DAS algorithm

### Problem setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Main system parameters: number of mics, number of samples in time
Nmics = 64
Nsamp = 200
# Source: x,y coordinates: x: 0, y: [-Y, +Y] where Y determined by pitch and Nmics
src = (0, 0)
# Spacing between microphones
pitch = 0.1
# proxy for sampling rate
dist_per_samp = 0.1
# Speed of sound in the medium
C = 2.0
# Time dilation factor for sinc pulse: how narrow
SincP = 5
# CODE Locations of microphones
mics = [(src[0],pitch*(2*i+1)/2) for i in range(-Nmics//2,Nmics//2)]
# Location of point obstacle
obstacle = (3,-1)

In [ ]:
t = 0 # CODE Nsamp time instants with spacing of dist_per_samp
def wsrc(t):
    return np.sinc(SincP*t)

### Prameter that changes sharpness of sinc pulse

In [ ]:
C=10
x=np.array([i*dist_per_samp/C for i in range(0,Nsamp)])
y=np.array([wsrc(x[i]) for i in range(0,Nsamp)])
plt.plot(x,y)

In [ ]:
C=2.0
x=np.array([i*dist_per_samp/C for i in range(0,Nsamp)])
y=np.array([wsrc(x[i]) for i in range(0,Nsamp)])
plt.plot(x,y)

The parameter that could be changed is C. For a given dist_per_samp if C is high time delay between samples is low. Thus the sinc pulse looks smeared or spread out. At low value of C it decays quite quickly. Thus `lower the value of C, higher is the sharpness of pulse`

### Distance traveled from the source to microphone with a reflection from a point

In [ ]:
def dist(src, pt, mic):
    d1 = ((src[0]-pt[0])**2+(src[1]-pt[1])**2)**0.5 # CODE distance from src to pt
    d2 = ((pt[0]-mic[0])**2+(pt[1]-mic[1])**2)**0.5 # CODE distance from pt to mic
    return d1 + d2

### Generating Mic Output

In [ ]:
C=2.0
# Define a list of delays for each mic
delays = [(dist(src, obstacle, mics[i])/C) for i in range(len(mics))]
y=[]
# Plot each curve with its corresponding delay
for i, delay in enumerate(delays):
    y+=[wsrc(x-delays[i])+(i-Nmics/2+0.5) * pitch]
    plt.plot(x,wsrc(x-delays[i])+(i-Nmics/2+0.5) * pitch)

plt.xlabel('X-Axis')
plt.ylabel('Data')
plt.show()
recons = np.zeros_like(y) #matrix used for reconstruction
for i, delay in enumerate(delays):
    recons[i] = y[i] - (i - Nmics/2 + 0.5) * pitch   

### Heatplot

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(recons, cmap='viridis', interpolation='nearest')
y=np.array(y)

### Delay-and-Sum algorithm

In [ ]:
# Assuming you have the necessary functions and parameters defined
# Create a grid of points
grid = [[(i * dist_per_samp, pitch * (2 * j + 1) / 2) for j in range(-Nmics // 2, Nmics // 2)] for i in range(Nsamp)]

# Initialize val with zeros
val = np.zeros(( Nmics,Nsamp))
max_value = 0
max_index = (0, 0)

# Iterate through each point in the grid
for i in range(Nsamp):
    for j in range(Nmics):
        v = 0
        # Iterate through each microphone
        for k in range(Nmics):
            # Accumulate the weighted source signal based on distance
            sampind = int(dist(src, grid[i][j], mics[k]) / dist_per_samp)
            if sampind<Nsamp:
                v += recons[k][sampind]
        # Assign the accumulated value to the corresponding position in val
        val[j][i] = v

        # Update max value and index
        if v >= max_value:
            max_value = v
            max_index = (i,j)

# Create a heatmap
fig, ax = plt.subplots()
im = ax.imshow(val,aspect='equal', cmap='viridis', interpolation='nearest')
plt.title('Delay-and-Sum Heatmap')
plt.colorbar(im)
plt.show()

In [ ]:
print(max_index)

## Explanation:

### Point upto which Reconstruction can take place
*It is not necessary to go upto Nsamp because even if iterate upto Nsamp only upto a condition when `total distance from source to grid point to mic is less than 20` (Nsamp* **dist_per_samp) we can use the value. After this, outer iterations of i is useless. This means that points with x_axis values above Nsamp/2 would never play a role. Iterating upto Nsamp/2 or even Nsamp* * *3/8 would be ideal*

### Reason for x,y coordinates
The obstacle was placed at (3,-1). 
* Y- axis:<br>
*The microphones that range from -3.15 to 3.15 are also shown from 0 to 63. This is because the pitch is 0.1 (3.15-(13.15)/ 0.1 =63).<br> Thus the point -1 is represented by -1-(-3.15)/0.1=21.5. The integer truncation has returned a value of 21*
* X axis:<br>
*The point 3 occurs after 3/dist_per_samp= 30 sampling points. Thus x_axis value is 30*

### Maximum x,y coordinates
The obstacle should be placed within a total distance from all source-obstacle-mics of 20 units of distance. It should thus lie definitely within radius of 10 units as we have trouble using samples above it.<br> With `y coordinates upto 3.15`, safe` x values are upto 9` (Though greater y coordinates could be considered we only sample points from -3.15 to 3.15 while plotting the grid for heatmap) 

### Sharpness of image:
The image becomes sharper if C is decreased because the wavelength becomes smaller as C decreses. Initially wavelength and pitch are equal. Then as wavelength gets much smaller than the dist_per_samp we get a better resolution<br>
This behavior aligns with the concept of the `Nyquist-Shannon sampling theorem`, which states that to accurately capture a signal, the sampling rate should be at least twice the frequency of the signal. In this case, `decreasing C effectively increases the frequency` (since the speed of sound is inversely proportional to the wavelength), leading to better resolution when the wavelength becomes much smaller than the sampling interval.

### Impact of Nmics and Nsamps
We observe that with increase in number of microphones, increase in samples increses resolution of heatmap.<br>
**Impact of Low Number of Microphones:** When the number of microphones is very low the obstacle cant be detected. This is because the y co-ordinate of obstacle is found using the fact that different microphones have different delays. If number of microphones is low, the have almost same delay. Thus only x-cordinate can be predicted. <br>**Resolution Increase with More Samples:** As number of samples is incresed for a given number of mics, its resolution increses<br> **Resolution Increase with More Microphones:** As the number of microphones is incresed its y coordinate can be predicted more accurately 

## 2 obstacles

In [ ]:
import csv
file_path = "C:\\Users\\shrip\\Downloads\\rx2.txt"
# Initialize an empty list to store the data
data = []
with open(file_path, 'r') as file:
    reader = csv.reader(file,delimiter=' ')
    for row in reader:
        data.append([float(value) for value in row])
data=np.array(data)

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(data)
C=0.5

In [ ]:
for i in range(len(data)):
    plt.plot(x,data[i]+(i-Nmics//2+0.5) * pitch)

In [ ]:
# Initialize val with zeros
val = np.zeros(( Nmics,Nsamp))
# Iterate through each point in the grid
for i in range(Nsamp):
    for j in range(Nmics):
        v = 0
        # Iterate through each microphone
        for k in range(Nmics):
            # Accumulate the weighted source signal based on distance
            sampind = int(dist(src, grid[i][j], mics[k]) / dist_per_samp)
            if sampind<Nsamp:
                v += data[k][sampind]

        # Assign the accumulated value to the corresponding position in val
        val[j][i] = v

# Create a heatmap
fig, ax = plt.subplots()
im = ax.imshow(val,aspect='equal', cmap='viridis', interpolation='nearest')
plt.title('Delay-and-Sum Heatmap')
plt.colorbar(im)
plt.show()


The obstacles appear to be placed at (2,0) and (3,1)

## 3 obstacles

In [ ]:
import csv
file_path = "C:\\Users\\shrip\\Downloads\\rx3.txt"
# Initialize an empty list to store the data
data1 = []
with open(file_path, 'r') as file:
    reader = csv.reader(file,delimiter=' ')
    for row in reader:
        data1.append([float(value) for value in row])
# Convert the data to a NumPy array
data1 = np.array(data1)

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(data1)

In [ ]:
for i in range(len(data1)):
    plt.plot(x,data1[i]+(i-Nmics//2+0.5) * pitch)

In [ ]:
# Initialize val with zeros
val = np.zeros(( Nmics,Nsamp))

# Iterate through each point in the grid
for i in range(Nsamp):
    for j in range(Nmics):
        v = 0
        # Iterate through each microphone
        for k in range(Nmics):
            # Accumulate the weighted source signal based on distance
            sampind = int(dist(src, grid[i][j], mics[k]) / dist_per_samp)
            if sampind<Nsamp:
                v += data1[k][sampind]

        # Assign the accumulated value to the corresponding position in val
        val[j][i] = v

# Create a heatmap
fig, ax = plt.subplots()
im = ax.imshow(val,aspect='equal', cmap='viridis', interpolation='nearest')
plt.title('Delay-and-Sum Heatmap')
plt.colorbar(im)
plt.show()

The obstacles appear to be placed at (2,0), (3,1) and (3.7,1)